In [ ]:
!pip install openimages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.1 MB/s eta 0:00:00


In [ ]:
import openimages

In [ ]:
from openimages.download import download_dataset
download_dataset("./", ["Human head"], annotation_format="darknet", limit=2000)

100%|██████████| 2000/2000 [00:00<00:00, 3112.96it/s]


{'human head': {'images_dir': './human head/images',
  'annotations_dir': './human head/darknet'}}

In [ ]:
!cat /content/human\ head/darknet/000004f4400f6ec5.txt

0 0.6184375 0.3244635 0.14812499999999995 0.290383


In [ ]:
!pip install ultralytics > /dev/null 2>&1

In [ ]:
nc =  1
names = ["Human head"]

with open("data.yaml", "w") as f:
    f.write(f"train: ../train/images\nval: ../valid/images\n\nnc: {nc}\nnames: {names}")

In [ ]:
!cat data.yaml

train: ../train/images
val: ../valid/images

nc: 1
names: ['Human head']

In [ ]:
!mv human\ head/darknet human\ head/labels

In [ ]:
import os
import random
import shutil
import yaml

DATASET_DIR = 'human head'
SAVE_SPLIT_DIR = './'

SIZE_VALID = 0.2

In [ ]:
path_train = os.path.join(SAVE_SPLIT_DIR, 'train')
os.mkdir(path_train)
path_train_images = os.path.join(path_train, 'images')
os.mkdir(path_train_images)
path_train_labels = os.path.join(path_train, 'labels')
os.mkdir(path_train_labels)

path_valid = os.path.join(SAVE_SPLIT_DIR, 'valid')
os.mkdir(path_valid)
path_valid_images = os.path.join(path_valid, 'images')
os.mkdir(path_valid_images)
path_valid_labels = os.path.join(path_valid, 'labels')
os.mkdir(path_valid_labels)

In [ ]:
path_images = os.path.join(DATASET_DIR, 'images')
path_labels = os.path.join(DATASET_DIR, 'labels')

In [ ]:
images = os.listdir(path_images)

valid_images = random.sample(images, int(len(images) * SIZE_VALID))
print(f'Size valid = {len(valid_images)}')
for filename in images:
    only_filename = filename[:-4]

    if filename in valid_images:
        shutil.copyfile(
            os.path.join(path_images, filename),
            os.path.join(path_valid_images, filename),
        )
        shutil.copyfile(
            os.path.join(path_labels, f'{only_filename}.txt'),
            os.path.join(path_valid_labels, f'{only_filename}.txt'),
        )
    else:
        shutil.copyfile(
            os.path.join(path_images, filename),
            os.path.join(path_train_images, filename),
        )
        shutil.copyfile(
            os.path.join(path_labels, f'{only_filename}.txt'),
            os.path.join(path_train_labels, f'{only_filename}.txt'),
        )

Size valid = 400


In [ ]:
import torch
import ultralytics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ultralytics.YOLO('yolo11n.yaml').to(device)

res = model.train(data='data.yaml', pretrained=True, epochs=100)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
engine/trainer: task=detect, mode=train, model=yolo11n.yaml, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visual

100%|██████████| 755k/755k [00:00<00:00, 26.4MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 82.1MB/s]


AMP: checks passed ✅


...

    100/100      2.48G      1.182     0.8289      1.266         36        640: 100%|██████████| 100/100 [00:32<00:00,  3.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:05<00:00,  2.57it/s]

                   all        400        978      0.666      0.583       0.56      0.332



100 epochs completed in 1.161 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.84 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:05<00:00,  2.26it/s]


                   all        400        978      0.654       0.58      0.557      0.336
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train
